# HR analytics

In [ ]:
>It is also known as people analytics.
>Is a data driven approach to manage peple at work.

# Problems addressed by HR

In [ ]:
> decisions related to employee hiring and retention.
> performance evaluation
> collaboration etc

# When skilled employees leave the company this can be very costly for the company thus firms are interested in predicting turnover before hand.

# Having that information in hand companies can change their strategies to retain good workers or start the hiring process of new employees on time.

In [2]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("E:\\aegis\\HRanalytics\\turnover.csv")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction            14999 non-null float64
evaluation              14999 non-null float64
number_of_projects      14999 non-null int64
average_montly_hours    14999 non-null int64
time_spend_company      14999 non-null int64
work_accident           14999 non-null int64
churn                   14999 non-null int64
promotion               14999 non-null int64
department              14999 non-null object
salary                  14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


# Finding categorical variables
Categorical variables are variables that receive a limited number of values that describe a category. They can be of two types:

Ordinal – variables with two or more categories that can be ranked or ordered (e.g. “low”, “medium”, “high”)
Nominal – variables with two or more categories that do not have an intrinsic order (e.g. “men”, “women”)
In this exercise, you will find the categorical variables in the dataset. To do that, first of all, you will import the pandas library and read the CSV file called "turnover.csv". Then, after viewing the first 5 rows and learning (visually) that there are non-numeric values in the DataFrame, you will get some information about the types of variables that are available in the dataset.

In [6]:
data.head()

,satisfaction,evaluation,number_of_projects,average_montly_hours,time_spend_company,work_accident,churn,promotion,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


# Observing categoricals
Remember from the previous exercise that:

Ordinal variables have two or more categories which can be ranked or ordered
Nominal variables have two or more categories which do not have an intrinsic order
In your dataset:

salary is an ordinal variable
department is a nominal variable
In this exercise, you're going to observe the categorical variables found in the previous exercise. To do that, first of all, you will import the pandas library and read the CSV file called "turnover.csv". Then, you will print the unique values of those variables

In [7]:
# Print the unique values of the "department" column
print(data.department.unique())

# Print the unique values of the "salary" column
print(data.salary.unique())

['sales' 'accounting' 'hr' 'technical' 'support' 'management' 'IT'
 'product_mng' 'marketing' 'RandD']
['low' 'medium' 'high']


# Encoding categories

In [8]:
# Change the type of the "salary" column to categorical
data.salary = data.salary.astype('category')

In [9]:
# Provide the correct order of categories
data.salary = data.salary.cat.reorder_categories(['low', 'medium', 'high'])

In [10]:
# Encode categories
data.salary = data.salary.cat.codes

# Our salary column is now encoded as an ordered category, and optimized for our prediction algorithm.

# Getting dummies
You will now transform the department variable, which you know is nominal based on the values you observed. To do that, you will use so-called dummy variables.

In [12]:
# Get dummies and save them inside a new DataFrame
departments = pd.get_dummies(data.department)

In [13]:
# Take a quick look to the first 5 rows of the new DataFrame called departments
print(departments.head())

   IT  RandD  accounting  hr  management  marketing  product_mng  sales  \
0   0      0           0   0           0          0            0      1   
1   0      0           0   0           0          0            0      1   
2   0      0           0   0           0          0            0      1   
3   0      0           0   0           0          0            0      1   
4   0      0           0   0           0          0            0      1   

   support  technical  
0        0          0  
1        0          0  
2        0          0  
3        0          0  
4        0          0  


# Dummy trap
A dummy trap is a situation where different dummy variables convey the same information. In this case, if an employee is, say, from the accounting department (i.e. value in the accounting column is 1), then you're certain that s/he is not from any other department (values everywhere else are 0). Thus, you could actually learn about his/her department by looking at all the other departments.

For that reason, whenever n dummies are created (in your case, 10), only n - 1 (in your case, 9) of them are enough, and the n-th column's information is already included.

Therefore, you will get rid of the old department column, drop one of the department dummies to avoid dummy trap, and then the two DataFrames.

In [14]:
# Drop the "accounting" column to avoid "dummy trap"
departments = departments.drop("accounting", axis=1)


In [15]:
# Drop the old column "department" as you don't need it anymore
data = data.drop("department", axis=1)


In [16]:
data.shape

(14999, 9)

In [17]:
# Join the new dataframe "departments" to your employee dataset: done
data = data.join(departments)

In [18]:
data.shape

(14999, 18)

# Percentage of employees who churn
The column churn is providing information about whether an employee has left the company or not is the column churn:

if the value of this column is 0, the employee is still with the company
if the value of this column is 1, then the employee has left the company
Let’s calculate the turnover rate:

you will first count the number of times the variable churn has the value 1 and the value 0, respectively
you will then divide both counts by the total, and multiply the result by 100 to get the percentage of employees who left and stayed

In [19]:
# Get the total number of observations and save it as the number of employees
n_employees = len(data)

# Print the number of employees who left/stayed
print(data.churn.value_counts())


0    11428
1     3571
Name: churn, dtype: int64


In [20]:
# Print the percentage of employees who left/stayed
print(data.churn.value_counts()/n_employees*100)

0    76.191746
1    23.808254
Name: churn, dtype: float64


# As you can see, 11,428 employees stayed, which accounts for about 76% of the total employee count. Similarly, 3,571 employees left, which accounts for about 24% of them

# Separating Target and Features
In order to make a prediction (in this case, whether an employee would leave or not), one needs to separate the dataset into two components:

the dependent variable or target which needs to be predicted
the independent variables or features that will be used to make a prediction
Your task is to separate the target and features. The target you have here is the employee churn, and features include everything else.

Reminder: the dataset has already been modified by encoding categorical variables and getting dummies.

pandas has been imported for you as pd.

In [21]:
# Choose the dependent variable column (churn) and set it as target
target = data.churn

# Drop column churn and set everything else as features
features = data.drop("churn",axis=1)

# Spliting employee data
Overfitting the dataset is a common problem in analytics. This happens when a model is working well on the dataset it was developed upon, but fails to generalize outside of it.

A train/test split is implemented to ensure model generalization: you develop the model using the training sample and try it out on the test sample later on.

In this exercise, you will split both target and features into train and test sets with 75%/25% ratio, respectively.

# Import the function for splitting dataset into train and test
from sklearn.model_selection import train_test_split

# Use that function to create the splits both for target and for features
# Set the test sample to be 25% of your observations
target_train, target_test, features_train, features_test = train_test_split(target,features,test_size=0.25,random_state=42)

# Computing Gini index
The decision tree algorithm aims to achieve partitions in the terminal nodes that are as pure as possible. The Gini index is one of the methods used to achieve this. It is calculated based on the proportion of samples in each group.

Given the number of people who stayed and left respectively, calculate the Gini index for that node.

In [23]:
#number of people who stayed/left
stayed = 37
left = 1138

#sum of stayed and left
total = stayed + left

#gini index
gini = 2*(stayed/total)*(left/total)

# Splitting the tree
Given the Gini index that would result from splitting by either variable A or B, respectively, decide by which variable the tree should split next.

In [24]:
# Gini index in case of splitting by variable A or B
gini_A = 0.65
gini_B = 0.15

# check which Gini is lower and use it for splitting
if gini_A < gini_B:
    print("split by A!")
else:
    print("split by B!")

split by B!


# Fitting the tree to employee data
A train/test split provides the opportunity to develop the classifier on the training component and test it on the rest of the dataset. In this exercise, you will start developing an employee turnover prediction model using the decision tree classification algorithm. The algorithm provides a .fit() method, which can be used to fit the features to the model in the training set.

Reminder: both target and features are already split into train and test components (Train: features_train, target_train, Test: features_test, target_test)

In [25]:
# Import the classification algorithm
from sklearn.tree import DecisionTreeClassifier

# Initialize it and call model by specifying the random_state parameter
model = DecisionTreeClassifier(random_state=42)

# Apply a decision tree model to fit features to the target
model.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

# Checking the accuracy of prediction
It’s now time to check how well your trained model can make predictions! Let’s use your testing set to check the accuracy of your Decision Tree model, with the score() method.

In [27]:
# Check the accuracy score of the prediction for the training set
model.score(features_train,target_train)*100



100.0

In [28]:
# Check the accuracy score of the prediction for the test set
model.score(features_test,target_test)*100

97.22666666666666

# As expected, your algorithm did perfectly on the training set. On the testing set, it was able to correctly predict if an employee would leave or not in almost 98% of the cases! Excellent!

# Exporting the tree
In Decision Tree classification tasks, overfitting is usually the result of deeply grown trees. As the comparison of accuracy scores on the train and test sets shows, you have overfitting in your results. This can also be learned from the tree visualization.

In this exercise, you will export the decision tree into a text document, which can then be used for visualization.

In [29]:
# Import the tree graphical visualization export function
from sklearn.tree import export_graphviz


In [30]:
# Apply Decision Tree model to fit Features to the Target
model.fit(features_train,target_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [31]:
# Export the tree to a dot file
export_graphviz(model,"tree.dot")

# Now you can copy the content of tree.dot file and preview it in webgraphviz.com!

# Pruning the tree
Overfitting is a classic problem in analytics, especially for the decision tree algorithm. Once the tree is fully grown, it may provide highly accurate predictions for the training sample, yet fail to be that accurate on the test set. For that reason, the growth of the decision tree is usually controlled by:

“Pruning” the tree and setting a limit on the maximum depth it can have.
Limiting the minimum number of observations in one leaf of the tree.
In this exercise, you will:

prune the tree and limit the growth of the tree to 5 levels of depth
fit it to the employee data
test prediction results on both training and testing sets.
The variables features_train, target_train, features_test and target_test are already available in your workspace.

In [32]:
# Initialize the DecisionTreeClassifier while limiting the depth of the tree to 5
model_depth_5 = DecisionTreeClassifier(max_depth=5, random_state=42)

# Fit and print the model
model_depth_5.fit(features_train,target_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [33]:
# Print the accuracy of the prediction for the training set
print(model_depth_5.score(features_train,target_train)*100)

# Print the accuracy of the prediction for the test set
print(model_depth_5.score(features_test,target_test)*100)

97.71535247577563
97.06666666666666


# Now you have a more reasonable model!

# Limiting the sample size
Another method to prevent overfitting is to specify the minimum number of observations necessary to grow a leaf (or node), in the Decision Tree.

In this exercise, you will:

set this minimum limit to 100
fit the new model to the employee data
examine prediction results on both training and test sets
The variables features_train, target_train, features_test and target_test are already available in your workspace.

# Calculating accuracy metrics: precision
The Precision score is an important metric used to measure the accuracy of a classification algorithm. It is calculated as the fraction of True Positives over the sum of True Positives and False Positives, or
# of True Positives# of True Positives+# of False Positives.
we define True Positives as the number of employees who actually left, and were classified correctly as leaving
we define False Positives as the number of employees who actually stayed, but were wrongly classified as leaving
If there are no False Positives, the precision score is equal to 1. If there are no True Positives, the recall score is equal to 0.

In this exercise, we will calculate the precision score (using the sklearn function precision_score) for our initial classification model.

The variables features_test and target_test are available in your workspace.

In [34]:
# Import the function to calculate precision score
from sklearn.metrics import precision_score

# Predict whether employees will churn using the test set
prediction = model.predict(features_test)

# Calculate precision score by comparing target_test with the prediction
precision_score(target_test, prediction)

0.9240641711229947

# Calculating accuracy metrics: recall
The Recall score is another important metric used to measure the accuracy of a classification algorithm. It is calculated as the** fraction of True Positives over the sum of True Positives and False Negatives**, or
# of True Positives# of True Positives+# of False Negatives.
If there are no False Negatives, the recall score is equal to 1. If there are no True Positives, the recall score is equal to 0.

In this exercise, you will calculate the precision score (using the sklearn function recall_score) for your initial classification model.

The variables features_test and target_test are available in your workspace

In [35]:
# Import the function to calculate recall score
from sklearn.metrics import recall_score

# Use the initial model to predict churn
prediction = model.predict(features_test)

# Calculate recall score by comparing target_test with the prediction
recall_score(target_test, prediction)

0.9632107023411371

# As the objective here is to develop a model that will correctly predict churn,recall score seems to be our target

# Calculating the ROC/AUC score
While the Recall score is an important metric for measuring the accuracy of a classification algorithm, it puts too much weight on the number of False Negatives. On the other hand, Precision is concentrated on the number of False Positives.

The combination of those two results in the ROC curve allows us to measure both recall and precision. The area under the ROC curve is calculated as the AUC score.

In this exercise, you will calculate the ROC/AUC score for the initial model using the sklearn roc_auc_score() function.

The variables features_test and target_test are available in your workspace.

In [36]:
# Import the function to calculate ROC/AUC score
from sklearn.metrics import roc_auc_score

# Use initial model to predict churn (based on features of the test set)
prediction = model.predict(features_test)

# Calculate ROC/AUC score by comparing target_test with the prediction
roc_auc_score(target_test, prediction)

0.9691623087590718

# Balancing classes
It can significantly affect prediction results, as shown by the difference between the recall and accuracy scores. To solve the imbalance, equal weights are usually given to each class. Using the class_weight argument in sklearn's DecisionTreeClassifier, one can make the classes become "balanced".

Let’s correct our model by solving its imbalance problem:

first, you’re going to set up a model with balanced classes
then, you will fit it to the training data
finally, you will check its accuracy on the test set
The variables features_train, target_train,featurestestandtargettest` are already available in your workspace.

In [38]:
# Initialize the DecisionTreeClassifier 
model_depth_5_b = DecisionTreeClassifier(max_depth=5,class_weight="balanced",random_state=42)

# Fit the model
model_depth_5_b.fit(features_train,target_train)

# Print the accuracy of the prediction (in percentage points) for the test set
print(model_depth_5_b.score(features_test,target_test)*100)

93.70666666666668


# Cross-validation using sklearn
As explained in Chapter 2, overfitting the dataset is a common problem in analytics. This happens when a model has learned the data too closely: it has great performances on the dataset it was trained on, but fails to generalize outside of it.

While the train/test split technique you learned in Chapter 2 ensures that the model does not overfit the training set, hyperparameter tuning may result in overfitting the test component, since it consists in tuning the model to get the best prediction results on the test set. Therefore, it is recommended to validate the model on different testing sets. K-fold cross-validation allows us to achieve this:

it splits the dataset into a training set and a testing set
it fits the model, makes predictions and calculates a score (you can specify if you want the accuracy, precision, recall...)
it repeats the process k times in total
it outputs the average of the 10 scores
In this exercise, you will use Cross Validation on our dataset, and evaluate our results with the cross_val_score functio

In [39]:
# Import the function for implementing cross validation
from sklearn.model_selection import cross_val_score

# Use that function to print the cross validation score for 10 folds
print(cross_val_score(model,features,target,cv=10))

[0.9853431  0.98533333 0.974      0.96533333 0.96       0.97933333
 0.99       0.99333333 1.         1.        ]


# Setting up GridSearch parameters
A hyperparameter is a parameter inside a function. For example, max_depth or min_samples_leaf are hyperparameters of the DecisionTreeClassifier() function. Hyperparameter tuning is the process of testing different values of hyperparameters to find the optimal ones: the one that gives the best predictions according to your objectives. In sklearn, you can use GridSearch to test different combinations of hyperparameters. Even better, you can use GridSearchCV() test different combinations and run cross-validation on them in one function!

In this exercise, you are going to prepare the different values you want to test for max_depth and min_samples_leaf. You will then put these in a dictionary, because that’s what is required for GridSearchCV():

the dictionary keys will be the hyperparameters names
the dictionary values will be the attributes (the hyperparameter values) you want to test
Instead of writing all the values manually, you will use the range() function, which allows us to generate values incrementally. For example, range(1, 10, 2) will generate a list containing values ranging from 1 included to 10 not included, by increments of 2. So the final result will be [1, 3, 5, 7, 9].



In [40]:
# Generate values for maximum depth
depth = [i for i in range(5,21,1)]

# Generate values for minimum sample size
samples = [i for i in range(50,500,50)]

# Create the dictionary with parameters to be checked
parameters = dict(max_depth=depth, min_samples_leaf=samples)

In [43]:
# import the GridSearchCV function
from sklearn.model_selection import GridSearchCV

# initialize the param_search function using the GridSearchCV function, initial model and parameters above
param_search = GridSearchCV(model, parameters)

# fit the param_search to the training dataset
param_search.fit(features_train, target_train)

# print the best parameters found
print(param_search.best_params_)


E:\ANACONDA\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 5, 'min_samples_leaf': 50}


# Feature Importance

# Sorting important features
Among other things, Decision Trees are very popular because of their interpretability. Many models can provide accurate predictions, but Decision Trees can also quantify the effect of the different features on the target. Here, it can tell you which features have the strongest and weakest impacts on the decision to leave the company. In sklearn, you can get this information by using the feature_importances_ attribute.

In this exercise, you're going to get the quantified importance of each feature, save them in a pandas DataFrame (a Pythonic table), and sort them from the most important to the less important. The model_ best Decision Tree Classifier used in the previous exercises is available in your workspace, as well as the features_test and features_train variables.

pandas has been imported as pd.

In [46]:
# Initialize the DecisionTreeClassifier 
model_best = DecisionTreeClassifier(max_depth=8,class_weight="balanced",min_samples_leaf=150,random_state=42)

# Fit the model
model_best.fit(features_train,target_train)

# Print the accuracy of the prediction (in percentage points) for the test set
print(model_best.score(features_test,target_test)*100)

95.28


In [47]:
# Calculate feature importances
feature_importances = model_best.feature_importances_


In [48]:
feature_importances

array([4.81864951e-01, 8.63953980e-02, 3.28493324e-02, 3.74608048e-02,
       3.58989477e-01, 0.00000000e+00, 0.00000000e+00, 4.45919858e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.91914039e-04,
       0.00000000e+00, 0.00000000e+00, 1.52528989e-04, 0.00000000e+00,
       1.54967392e-03])

In [49]:
# Create a list of features: done
feature_list = list(features)

In [50]:
# Save the results inside a DataFrame using feature_list as an indnex
relative_importances = pd.DataFrame(index=feature_list, data=feature_importances, columns=["importance"])

In [51]:
relative_importances

,importance
satisfaction,0.481865
evaluation,0.086395
number_of_projects,0.032849
average_montly_hours,0.037461
time_spend_company,0.358989
work_accident,0.000000
promotion,0.000000
salary,0.000446
IT,0.000000
RandD,0.000000


In [52]:
# Sort the DataFrame to learn most important features
relative_importances.sort_values(by="importance", ascending=False)

,importance
satisfaction,0.481865
time_spend_company,0.358989
evaluation,0.086395
average_montly_hours,0.037461
number_of_projects,0.032849
technical,0.001550
salary,0.000446
management,0.000292
sales,0.000153
promotion,0.000000


# Selecting important features
In this exercise, your task is to select only the most important features that will be used by the final model. Remember, that the relative importances are saved in the column importance of the DataFrame called relative_importances.

In [53]:
# select only features with relative importance higher than 1%
selected_features = relative_importances[relative_importances.importance>0.01]

In [54]:
# create a list from those features: done
selected_list = selected_features.index

In [55]:
selected_list

Index(['satisfaction', 'evaluation', 'number_of_projects',
       'average_montly_hours', 'time_spend_company'],
      dtype='object')

In [56]:
# transform both features_train and features_test components to include only selected features
features_train_selected = features_train[selected_list]
features_test_selected = features_test[selected_list]

# Develop and test the best model
In Chapter 3, you found out that the following parameters allow you to get better model:

max_depth = 8,
min_samples_leaf = 150,
class_weight = "balanced"
In this chapter, you discovered that some of the features have a negligible impact. You realized that you could get accurate predictions using just a small number of selected, impactful features and you updated your training and testing set accordingly, creating the variables features_train_selected and features_test_selected.

With all this information at your disposal, you're now going to develop the best model for predicting employee turnover and evaluate it using the appropriate metrics.

The features_train_selected and features_test_selected variables are available in your workspace, and the recall_score and roc_auc_score functions have been imported for you.

In [57]:
# Initialize the best model using parameters provided in description
model_best = DecisionTreeClassifier(max_depth=8, min_samples_leaf=150, class_weight="balanced", random_state=42)

# Fit the model using only selected features from training set: done
model_best.fit(features_train_selected, target_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=150, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [58]:
# Make prediction based on selected list of features from test set
prediction_best = model_best.predict(features_test_selected)

# Print the general accuracy of the model_best
print(model_best.score(features_test_selected, target_test) * 100)


95.28


In [59]:
# Print the recall score of the model predictions
print(recall_score(target_test, prediction_best) * 100)

91.75027870680044


In [60]:
# Print the ROC/AUC score of the model predictions
print(roc_auc_score(target_test, prediction_best) * 100)

94.07002193314084
